# Load cleaned banking CSV files into DuckDB tables for analysis

In [70]:
import duckdb

con = duckdb.connect()

base_path = "/Users/mac/Desktop/Banking_project/raw_data_clean"

con.execute(f"""
CREATE OR REPLACE TABLE customers AS 
SELECT * FROM read_csv_auto('{base_path}/bank_customers_cleaned.csv');
""")

con.execute(f"""
CREATE OR REPLACE TABLE accounts AS 
SELECT * FROM read_csv_auto('{base_path}/bank_accounts_cleaned.csv');
""")

con.execute(f"""
CREATE OR REPLACE TABLE loans AS 
SELECT * FROM read_csv_auto('{base_path}/bank_loans_cleaned.csv');
""")

con.execute(f"""
CREATE OR REPLACE TABLE transactions AS 
SELECT * FROM read_csv_auto('{base_path}/bank_transactions_cleaned.csv');
""")

con.execute(f"""
CREATE OR REPLACE TABLE complaints AS 
SELECT * FROM read_csv_auto('{base_path}/bank_complaints_cleaned.csv');
""")



# Overviwe Analysis Page with 10 KPI's

###  Total number of unique customers


In [71]:
ov_kpi_1 = con.execute("""
    SELECT COUNT(DISTINCT customer_id) AS no_of_customers 
    FROM customers
                    """).df


###  Total number of unique bank accounts


In [72]:
ov_kpi_2 = con.execute("""
select count(distinct account_id) as NO_OF_ACC from accounts
                    """).df()

###  Total number of unique transactions


In [73]:
ov_kpi_3 = con.execute(""" 
select count(distinct transaction_id ) as NO_OF_TRANSACTIONS from transactions
                    """).df()

###  Total transaction amount (rounded to 2 decimals)


In [74]:
ov_kpi_4 = con.execute("""
select round(sum(amount),2) as TOTAL_AMOUNT from transactions
                    """).df()

###  Average transaction amount (rounded to 2 decimals)


In [75]:
ov_kpi_5 = con.execute("""
select round(avg(amount),2) as AVG_TRANS_AMOUNT from transactions
                    """).df()

###  Percentage of defaulted loans (rounded to 2 decimals)


In [76]:
ov_kpi_6 = con.execute(""" 
select round(
(select count(*) from loans where loan_status = 'Defaulted') / count(*) * 100 
             ,2)
as DEF__LOANS_PERCENT from loans 
                    """).df()

###  Percentage of customers who submitted complaints


In [77]:
ov_kpi_7 = con.execute(""" 

select round( 
 count(distinct customer_id) / 
(select count(*) from customers ) * 100 
      ,2)
AS COMPLAINTS_CUS_RATIO 
from complaints  
""").df()

###  Top 5 cities by number of customers


In [78]:
ov_kpi_8 = con.execute(""" 
select city , count(customer_id) as NO_OF_CUSTOMERS 
from customers
group by city 
order by count(customer_id) desc 
limit 5 
""").df()

###  Number of accounts by account type


In [79]:
ov_kpi_9 = con.execute(""" 
select account_type , count(account_id) as NO_OF_ACCOUNTS 
from accounts 
group by account_type
order by count(account_id) desc 
""").df() 

###  Number of complaints by status


In [80]:
ov_kpi_10 = con.execute(""" 
SELECT status, COUNT(complaint_id) AS no_of_complaints 
FROM complaints 
GROUP BY status 
ORDER BY COUNT(complaint_id) DESC
""").df()


###  Export overview KPIs to CSV from 1 to 7 

In [81]:
import pandas as pd

def extract_value(kpi):
    if hasattr(kpi, 'iloc'):
        return kpi.iloc[0, 0]
    return kpi

kpi_values = [
    ("NO_OF_CUSTOMERS", extract_value(ov_kpi_1)),
    ("NO_OF_ACCOUNTS", extract_value(ov_kpi_2)),
    ("NO_OF_TRANSACTIONS", extract_value(ov_kpi_3)),
    ("TOTAL_TRANSACTION_AMOUNT", extract_value(ov_kpi_4)),
    ("AVG_TRANS_AMOUNT", extract_value(ov_kpi_5)),
    ("DEF_LOANS_PERCENT", extract_value(ov_kpi_6)),
    ("COMPLAINTS_CUS_RATIO", extract_value(ov_kpi_7)),
]

df = pd.DataFrame(kpi_values, columns=["KPI_NAME", "VALUE"])

df.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/overview_summary_kpis.csv", index=False)


###  Export overview KPIs to CSV from 8 to 10 

In [82]:
ov_kpi_8.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/overview_top5_cities.csv", index=False)

ov_kpi_9.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/overview_account_types_distribution.csv", index=False)

ov_kpi_10.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/overview_complaints_status_distribution.csv", index=False)


# Customers Analysis with 10 KPI's 

###  Average age of customers (rounded to 2 decimals)


In [83]:
cus_kpi_1 = con.execute(""" 
select round(avg(age),2) as AVG_CUSTOMERS_AGEs from customers 
""").df()

###  Number of customers by marital status


In [84]:
cus_kpi_2 = con.execute(""" 
select Marital_Status , count(customer_id) as NO_OF_CUSTOMERS from customers
group by Marital_Status 
order by count(customer_id) desc
""").df()

###  Top 5 customer jobs by count


In [85]:
cus_kpi_3 = con.execute(""" 
select job , count(customer_id) as NO_OF_CUSTOMERS from customers 
group by job 
order by count(customer_id) desc
limit 5 
""").df()

###  Number of customers by city


In [86]:
cus_kpi_4 = con.execute(""" 
select city , count(customer_id) as NO_OF_CUSTOMERS from customers 
group by city 
order by count(customer_id) desc 
""").df()

###  Number of customers with at least one loan


In [87]:
cus_kpi_5 = con.execute(""" 
select count(distinct customer_id) as NO_OF_LOANED_CUSTOMERS from loans 
""").df() 

###  Percentage of customers with loans


In [88]:
cus_kpi_7 = con.execute(""" 

select round( 
 count(distinct customer_id) / 
(select count(*) from customers ) * 100 
      ,2)
AS loaned_CUS_RATIO 
from loans
""").df()

###  Average loan amount by customer age group


In [89]:
cus_kpi_8 = con.execute("""
SELECT 
    age_group,
    ROUND(AVG(loan_amount), 2) AS avg_loan_amount
FROM (
    SELECT 
        CONCAT(FLOOR(c.age / 10) * 10, '-', FLOOR(c.age / 10) * 10 + 9) AS age_group,
        l.loan_amount
    FROM customers c
    JOIN loans l ON c.customer_id = l.customer_id
) AS age_bins
GROUP BY age_group
ORDER BY age_group
""").df()


###  City with the highest number of defaulted loans


In [90]:
cus_kpi_9 = con.execute("""
select c.city , count(l.loan_id) as MAX_LOANS 
from customers as c join loans as l 
on c.customer_id = l.customer_id 
where l.loan_status = 'Defaulted' 
group by c.city 
order by count(l.loan_id) desc 
limit 1
""").df() 

###  Sample of customers with accounts, loans, and complaints


In [91]:
cus_kpi_10 = con.execute("""
SELECT 
  c.customer_id, c.age,c.city, c.marital_status, c.job
FROM customers c
JOIN accounts as a ON c.customer_id = a.customer_id
JOIN loans as l ON c.customer_id = l.customer_id
JOIN complaints as cmp ON c.customer_id = cmp.customer_id
LIMIT 10
""").df() 


###  Export customer summary KPIs to CSV 1,5,7,10 


In [92]:
def extract_value(kpi):
    if hasattr(kpi, 'iloc'):
        return kpi.iloc[0, 0]
    return kpi

customer_summary = [
    ("AVG_CUSTOMERS_AGE", extract_value(cus_kpi_1)),
    ("NO_OF_LOANED_CUSTOMERS", extract_value(cus_kpi_5)),
    ("LOANED_CUS_RATIO", extract_value(cus_kpi_7)),
    ("CUSTOMERS_WITH_ACC_LOAN_COMP", extract_value(cus_kpi_10))
]

df_customers_summary = pd.DataFrame(customer_summary, columns=["KPI_NAME", "VALUE"])
df_customers_summary.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_summary_kpis.csv", index=False)


###  Export customer  KPIs to CSV 2,3,4,8,9 


In [93]:
cus_kpi_2.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_marital_status_distribution.csv", index=False)

cus_kpi_3.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_top5_jobs.csv", index=False)

cus_kpi_4.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_by_city.csv", index=False)

cus_kpi_8.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_avg_loan_by_age_group.csv", index=False)

cus_kpi_9.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_city_with_max_default_loans.csv", index=False)


# Loans Analysis with 8 KPI's 

###  Total number of loans


In [94]:
loa_kpi_1 = con.execute(""" 
select count(distinct loan_id) as NO_OF_LOANS from loans
""").df()

###  Number of defaulted loans per 1000 loans


In [95]:
loa_kpi_2 = con.execute(""" 
select round( count(distinct loan_id) / (select count(distinct loan_id) from loans) * 1000 ,2) 
as Ratio_Def_Loans from loans where loan_status = 'Defaulted' 
""").df() 

###  Average loan amount (rounded to 2 decimals)


In [96]:
loa_kpi_3 = con.execute(""" 
select round(avg(loan_amount),2) from loans  
""").df() 

###  Number of loans by loan type


In [97]:
loa_kpi_4 = con.execute(""" 
select loan_type , count(loan_id) from loans group by loan_type order by count(loan_id) desc
""").df()

###  Number of loans by loan status


In [98]:
loa_kpi_5 = con.execute(""" 
select loan_status , count(loan_id) from loans group by loan_status order by count(loan_id) desc
""").df()

###  Number of customers with loans by city


In [99]:
loa_kpi_6 = con.execute(""" 
select c.city , count(l.customer_id) as NO_OF_CUSTOMERS 
from customers as c join loans as l 
on c.customer_id = l.customer_id 
group by c.city 
order by count(l.customer_id) desc
""").df() 

###  Customers with more than one loan


In [100]:
loa_kpi_7 = con.execute("""
select  c.customer_id,c.name , count(l.loan_id) as NO_OF_LOANS from customers as c join loans as l 
on c.customer_id = l.customer_id 
group by c.name , c.customer_id
having count(l.loan_id) > 1 
 """).df()

###  Number of defaulted loans by loan type


In [101]:
loa_kpi_8 = con.execute("""
SELECT 
    loan_type,
    COUNT(*) AS num_defaulted_loans
FROM loans
WHERE loan_status = 'Defaulted'
GROUP BY loan_type
ORDER BY COUNT(*) DESC
""").df()


###  Export loan summary KPIs to CSV 1,2,3


In [102]:
def extract_value(kpi):
    if hasattr(kpi, 'iloc'):
        return kpi.iloc[0, 0]
    return kpi

loans_summary = [
    ("NO_OF_LOANS", extract_value(loa_kpi_1)),
    ("RATIO_DEF_LOANS", extract_value(loa_kpi_2)),
    ("AVG_LOAN_AMOUNT", extract_value(loa_kpi_3))
]

df_loans_summary = pd.DataFrame(loans_summary, columns=["KPI_NAME", "VALUE"])
df_loans_summary.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/loans_summary_kpis.csv", index=False)


###  Export loan  KPIs to CSV 4,5,6,7,8


In [103]:
loa_kpi_4.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/loans_by_type.csv", index=False)

loa_kpi_5.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/loans_by_status.csv", index=False)

loa_kpi_6.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/defaulted_customers_summary.csv", index=False)

loa_kpi_7.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_with_multiple_loans.csv", index=False)

loa_kpi_8.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/loans_defaulted_by_type.csv", index=False)


#  Complaints & Customer Feedback Analysis 8 KPI's

###  Total number of complaints


In [104]:
com_kpi_1 = con.execute("""
select count(complaint_id) as NO_OF_COMPLAINTS from complaints
""").df()

###  Percentage of customers who submitted complaints


In [105]:
com_kpi_2 = con.execute(""" 
select round(count(distinct customer_id ) / (select count(customer_id) from customers ) * 100 ,2) 
as COMP_CUS_RATIO from complaints
""").df()

###  Number of complaints by status


In [106]:
com_kpi_3 = con.execute("""
select status , count(complaint_id) as NO_OF_COMPLAINTS
from complaints 
group by status 
order by count(complaint_id) desc
""").df()

###  Number of complaints by city


In [107]:
com_kpi_4 = con.execute("""
select c.city , count(com.complaint_id) as NO_OF_COMPLAINTS 
from customers as c join complaints as com 
on c.customer_id = com.customer_id 
group by c.city 
order by count(com.complaint_id) desc 
""").df()

###  Number of complaints by customer job


In [108]:
com_kpi_5 = con.execute("""
select c.job , count(com.complaint_id) as NO_OF_COMPLAINTS 
from customers as c join complaints as com 
on c.customer_id = com.customer_id
group by c.job 
order by count(com.complaint_id) 
""").df()

###  Percentage of complaints from customers with loans


In [109]:
 com_kpi_6 = con.execute("""
SELECT 
    ROUND(
        (
            SELECT COUNT(DISTINCT cmp.customer_id)
            FROM complaints cmp
            JOIN loans l ON cmp.customer_id = l.customer_id
        ) * 100.0
        /
        (SELECT COUNT(DISTINCT customer_id) FROM complaints),
    2
    ) AS complaint_from_loan_customers_ratio
""").df()

###  Number of customers with multiple complaints


In [110]:
com_kpi_7 = con.execute("""
SELECT COUNT(*) AS multi_complaint_customers
FROM (
    SELECT customer_id
    FROM complaints
    GROUP BY customer_id
    HAVING COUNT(complaint_id) > 1
) AS sub
""").df()


###  Monthly complaint count by status


In [111]:
com_kpi_8 = con.execute("""
SELECT 
    STRFTIME(complaint_date, '%Y-%m') AS month,
    status,
    COUNT(*) AS no_of_complaints
FROM complaints
GROUP BY month, status
ORDER BY month
""").df()


###  Export complaint KPIs (1,2,6,7)


In [112]:
def extract_value(kpi):
    return kpi.iloc[0, 0] if hasattr(kpi, 'iloc') else kpi


complaints_summary = [
    ("NO_OF_COMPLAINTS", extract_value(com_kpi_1)),
    ("COMPLAINTS_CUSTOMERS_RATIO", extract_value(com_kpi_2)),
    ("LOAN_CUSTOMERS_COMPLAINT_RATIO", extract_value(com_kpi_6)),
    ("MULTI_COMPLAINT_CUSTOMERS", extract_value(com_kpi_7)),
]
df_complaints_summary = pd.DataFrame(complaints_summary, columns=["KPI_NAME", "VALUE"])
df_complaints_summary.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/complaints_summary_kpis.csv", index=False)


###  Export complaint KPIs (3,4,5,8)


In [113]:
com_kpi_3.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/complaints_by_status.csv", index=False)
com_kpi_4.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/complaints_by_city.csv", index=False)
com_kpi_5.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/complaints_by_job.csv", index=False)
com_kpi_8.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/complaints_trend_by_month.csv", index=False)

# Account & Transaction Insights

###  Total number of unique bank accounts


In [114]:
acc_kpi_1 = con.execute("""
SELECT COUNT(DISTINCT account_id) AS no_of_accounts
FROM accounts
""").df()


###  Number of accounts by account type


In [115]:
acc_kpi_2 = con.execute("""
SELECT account_type, COUNT(*) AS no_of_accounts
FROM accounts
GROUP BY account_type
ORDER BY COUNT(*) DESC
""").df()


###  Average account age in years


In [116]:
acc_kpi_3 = con.execute("""
SELECT 
  ROUND(AVG(DATE_DIFF('day', open_date, CURRENT_DATE)) / 365.25, 1) AS avg_account_age_years
FROM accounts
""").df()

###  Total number of transactions


In [117]:
acc_kpi_4 = con.execute("""
SELECT COUNT(transaction_id) AS no_of_transactions
FROM transactions
""").df()


###  Average transaction amount (rounded to 2 decimals)


In [118]:
acc_kpi_5 = con.execute("""
SELECT ROUND(AVG(amount), 2) AS avg_transaction_amount
FROM transactions
""").df()


###  Number of transactions by transaction type


In [119]:
acc_kpi_6 = con.execute("""
SELECT transaction_type, COUNT(*) AS no_of_transactions
FROM transactions
GROUP BY transaction_type
ORDER BY COUNT(*) DESC
""").df()


###  Top 5 cities by number of transactions


In [120]:
acc_kpi_7 = con.execute("""
SELECT c.city, COUNT(t.transaction_id) AS no_of_transactions
FROM customers c
JOIN accounts a ON c.customer_id = a.customer_id
JOIN transactions t ON a.account_id = t.account_id
GROUP BY c.city
ORDER BY COUNT(t.transaction_id) DESC
LIMIT 5
""").df()


###  Customers with more than 20 transactions


In [121]:
acc_kpi_8 = con.execute("""
SELECT c.customer_id, c.name, COUNT(t.transaction_id) AS total_transactions
FROM customers c
JOIN accounts a ON c.customer_id = a.customer_id
JOIN transactions t ON a.account_id = t.account_id
GROUP BY c.customer_id, c.name
HAVING COUNT(t.transaction_id) > 20
ORDER BY total_transactions DESC
""").df()


###  Top 5 customers by total transaction amount


In [122]:
acc_kpi_9 = con.execute("""
SELECT c.customer_id, c.name, ROUND(SUM(t.amount), 2) AS total_spent
FROM customers c
JOIN accounts a ON c.customer_id = a.customer_id
JOIN transactions t ON a.account_id = t.account_id
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC
LIMIT 5
""").df()


###  Most frequent transaction type


In [123]:
acc_kpi_10 = con.execute("""
SELECT transaction_type, COUNT(*) AS no_of_transactions
FROM transactions
GROUP BY transaction_type
ORDER BY COUNT(*) DESC
LIMIT 1
""").df()


###  Export account and transaction summary KPIs


In [124]:
acc_summary = [
    ("NO_OF_ACCOUNTS", acc_kpi_1.iloc[0, 0]),
    ("AVG_ACCOUNT_AGE_YEARS", acc_kpi_3.iloc[0, 0]),
    ("NO_OF_TRANSACTIONS", acc_kpi_4.iloc[0, 0]),
    ("AVG_TRANSACTION_AMOUNT", acc_kpi_5.iloc[0, 0]),
    ("MOST_COMMON_TRANSACTION_TYPE", acc_kpi_10.iloc[0, 0])
]

df_acc_summary = pd.DataFrame(acc_summary, columns=["KPI_NAME", "VALUE"])
df_acc_summary.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/accounts_summary_kpis.csv", index=False)


###  Export account and transactions (2,6,7,8,9)


In [125]:

acc_kpi_2.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/accounts_by_type.csv", index=False)
acc_kpi_6.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/transactions_by_type.csv", index=False)
acc_kpi_7.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/transactions_by_city.csv", index=False)
acc_kpi_8.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/customers_with_high_transaction_volume.csv", index=False)
acc_kpi_9.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/top_spending_customers.csv", index=False)


# Geo Analysis 

###  Number of customers by city


In [126]:
geo_kpi_1 = con.execute("""
SELECT city, COUNT(*) AS no_of_customers
FROM customers
GROUP BY city
ORDER BY COUNT(*) DESC
""").df()


###  Number of loans by city


In [127]:
geo_kpi_2 = con.execute("""
SELECT c.city, COUNT(l.loan_id) AS no_of_loans
FROM customers c
JOIN loans l ON c.customer_id = l.customer_id
GROUP BY c.city
ORDER BY COUNT(l.loan_id) DESC
""").df()


###  Loan default rate by city (in %)


In [128]:
geo_kpi_3 = con.execute("""
SELECT 
    c.city,
    ROUND(
        SUM(CASE WHEN l.loan_status = 'Defaulted' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 
        2
    ) AS default_rate
FROM customers c
JOIN loans l ON c.customer_id = l.customer_id
GROUP BY c.city
ORDER BY default_rate DESC
""").df()


###  Number of complaints by city


In [129]:
geo_kpi_4 = con.execute("""
SELECT c.city, COUNT(cmp.complaint_id) AS no_of_complaints
FROM customers c
JOIN complaints cmp ON c.customer_id = cmp.customer_id
GROUP BY c.city
ORDER BY COUNT(cmp.complaint_id) DESC
""").df()


###  Export geographic KPIs (customers, loans, defaults, complaints by city)


In [130]:

geo_kpi_1.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/geo_customers_by_city.csv", index=False)
geo_kpi_2.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/geo_loans_by_city.csv", index=False)
geo_kpi_3.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/geo_default_rate_by_city.csv", index=False)
geo_kpi_4.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/geo_complaints_by_city.csv", index=False)


# Trend Analysis

### Monthly trend of transaction count and total amount


In [131]:
trend_kpi_1 = con.execute("""
SELECT 
    STRFTIME(transaction_date, '%Y-%m') AS month,
    COUNT(*) AS no_of_transactions,
    ROUND(SUM(amount), 2) AS total_transaction_amount
FROM transactions
GROUP BY month
ORDER BY month
""").df()

###  Monthly complaint trend by status


In [132]:
trend_kpi_2 = con.execute("""
SELECT 
    STRFTIME(complaint_date, '%Y-%m') AS month,
    status,
    COUNT(*) AS no_of_complaints
FROM complaints
GROUP BY month, status
ORDER BY month
""").df()


###  Export monthly trend KPIs (loans and complaints by status)


In [133]:
trend_kpi_1.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/trend_monthly_transactions.csv", index=False)
trend_kpi_2.to_csv("/Users/mac/Desktop/Banking_project/kpi_exports/trend_monthly_complaints_by_status.csv", index=False)